<a href="https://colab.research.google.com/github/lukasbelka/datenanalyse-beispiel/blob/main/Datenanalyse_Beipsiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lukasbelka/datenanalyse-beispiel.git

Cloning into 'datenanalyse-beispiel'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 456.31 KiB | 19.01 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
"/content/datenanalyse-beispiel/produkt-verkaeufe.csv"